In [1]:
  #!/usr/bin/env python
#Numcosmo import
try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
import sys

# General imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.table import Table
from astropy.io import fits
import pandas as pd

# DC2 catalog-related imports
import FoFCatalogMatching
import GCRCatalogs
from GCR import GCRQuery

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

In [2]:
#A list with all DC2 catalogs available

#GCRCatalogs.get_available_catalog_names(include_default_only=False)

In [3]:
#We are loading the cosmoDC2_redmapper catalalog with richness
redmapper_cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')

#Taking some important information about the fiducial cosmological model and survey region
redmapper_cosmology = redmapper_cat.cosmology
sky_area = redmapper_cat.sky_area

#Listing all quantities in the choosen catalog
np.sort(redmapper_cat.list_all_quantities())

array(['cluster_id', 'cluster_id_member', 'dec', 'dec_cen_0', 'dec_cen_1',
       'dec_cen_2', 'dec_cen_3', 'dec_cen_4', 'dec_member', 'id_cen_0',
       'id_cen_1', 'id_cen_2', 'id_cen_3', 'id_cen_4', 'id_member',
       'mag_g_lsst_member', 'mag_i_lsst_member', 'mag_r_lsst_member',
       'mag_y_lsst_member', 'mag_z_lsst_member', 'magerr_g_lsst_member',
       'magerr_i_lsst_member', 'magerr_r_lsst_member',
       'magerr_y_lsst_member', 'magerr_z_lsst_member', 'maskfrac',
       'p_cen_0', 'p_cen_1', 'p_cen_2', 'p_cen_3', 'p_cen_4', 'p_member',
       'pfree_member', 'ra', 'ra_cen_0', 'ra_cen_1', 'ra_cen_2',
       'ra_cen_3', 'ra_cen_4', 'ra_member', 'redshift', 'redshift_err',
       'redshift_true_cg', 'redshift_true_member', 'refmag_err_member',
       'refmag_member', 'richness', 'richness_err', 'scaleval',
       'theta_i_member', 'theta_r_member'], dtype='<U20')

In [4]:
z_min = 0
z_max = 1
richness_min = 0
richness_max = 200
data = redmapper_cat.get_quantities(['cluster_id','redshift','redshift_err','richness','richness_err','ra','dec'],
                                   filters = [f'redshift > {z_min}',f'redshift < {z_max}',f'richness > {richness_min}',f'richness < {richness_max}' ])


redshift_err = data['redshift_err']
richness = data['richness']
richness_err = data['richness_err']
redshift = data['redshift']
ra = data['ra']
dec = data['dec']
cluster_id = data['cluster_id']

ncountdata = {'cluster_id': cluster_id, 'richness': richness, 'richness_err': richness_err,'z': redshift,'z_err': redshift_err,'ra':ra,'dec':dec}
ncountdata = pd.DataFrame(ncountdata)


N_cl = len(ncountdata)
print(f'There are {N_cl} halos with Richness= [{richness_min:.2e},{richness_max:.2e}] and z=[{z_min},{z_max}] in {redmapper_cat.sky_area} deg2')
display(ncountdata)

There are 38699 halos with Richness= [0.00e+00,2.00e+02] and z=[0,1] in 439.78987 deg2


,cluster_id,richness,richness_err,z,z_err,ra,dec
0,10,190.706650,3.746967,0.513513,0.002786,52.520436,-25.717996
1,18,179.128128,3.693436,0.571270,0.003218,56.920377,-27.204341
2,30,167.336029,3.677716,0.635185,0.006033,58.563624,-27.429962
3,71,111.914703,3.366554,0.442303,0.003674,58.355198,-26.867687
4,73,104.439766,2.957060,0.466972,0.003501,57.765757,-28.119102
...,...,...,...,...,...,...,...
38694,92281,5.572279,1.504350,0.594197,0.014232,68.391998,-45.515891
38695,35682,5.938398,1.263046,0.471931,0.006561,69.518360,-44.856106
38696,54206,5.282434,1.234711,0.501281,0.008677,70.472970,-44.819294
38697,97912,5.007845,0.897868,0.211521,0.006237,71.592885,-44.833270


# Numcosmo Cosmology

In [5]:
cosmo = Nc.HICosmoDEXcdm()
cosmo.props.H0      = redmapper_cosmology.H0.value
cosmo.props.Omegab  = redmapper_cosmology.Ob0
cosmo.props.Omegac  = redmapper_cosmology.Odm0
cosmo.props.Omegax  = redmapper_cosmology.Ode0

reion = Nc.HIReionCamb.new () 
prim = Nc.HIPrimPowerLaw.new () 

cosmo.add_submodel (reion)
cosmo.add_submodel (prim)

dist = Nc.Distance.new (2.0)

tf = Nc.TransferFunc.new_from_name ("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new (tf)
psml.require_kmin (1.0e-6)
psml.require_kmax (1.0e3)

psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0 ()

mulf = Nc.MultiplicityFuncBocquet.new ()
mulf.set_mdef (Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta (200.0)
mulf.set_sim (Nc.MultiplicityFuncBocquetSim.DM)

hmf = Nc.HaloMassFunction.new (dist, psf, mulf)
hmf.set_area(sky_area)

In [6]:
cluster_z = Nc.ClusterRedshift.new_from_name("NcClusterPhotozGauss{'pz-min': <%20.15e>, 'pz-max':<%20.15e>}" % (z_min,z_max))
cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}" % (3e14/(0.71),0.6, richness_min, richness_max))
cluster_m.param_set_by_name('mup0', 3.19)
cluster_m.param_set_by_name('mup1', 2/np.log(10))
cluster_m.param_set_by_name('mup2', -0.7/np.log(10))
cluster_m.param_set_by_name('sigmap0', 0.33)
cluster_m.param_set_by_name('sigmap1', -0.08/np.log(10))
cluster_m.param_set_by_name('sigmap2', 0/np.log(10))

# Preparing the fit with Numcosmo

In [10]:
ca = Nc.ClusterAbundance.new(hmf,None)
mset = Ncm.MSet.new_array([cosmo,cluster_m,cluster_z])
ncdata = Nc.DataClusterNCount.new(ca,'NcClusterPhotozGauss','NcClusterMassAscaso')






z_obs_params = Ncm.Matrix.new(len(redshift_err),2)
for i in range(len(redshift_err)):
    for j in range(0,2):
        if j == 0:
            z_obs_params.set(i,j,redshift_err[i])
        if j == 1:
            z_obs_params.set(i,j,0)

ncdata.set_lnM_obs (Ncm.Matrix.new_array (richness,1))
ncdata.set_z_obs (Ncm.Matrix.new_array (redshift,1))
ncdata.set_z_obs_params (z_obs_params)

ncdata.true_data (False)
ncdata.set_init(True)

#Bins definition
z_edges = np.linspace(z_min, z_max, 5)
proxy_edges = np.linspace(richness_min, richness_max, 4)

ncdata.del_bins ()
for rich_l, rich_u in zip(proxy_edges, proxy_edges[1:]):
    for zl, zu in zip(z_edges,z_edges[1:]):
        lnM_obs_lb = Ncm.Vector.new_array ([rich_l])
        lnM_obs_ub = Ncm.Vector.new_array ([rich_u])
        z_obs_lb = Ncm.Vector.new_array ([zl])
        z_obs_ub = Ncm.Vector.new_array ([zu])
        
        ncdata.add_bin (lnM_obs_lb, lnM_obs_ub, z_obs_lb, z_obs_ub)

ncdata.bin_data ()

ncdata.catalog_save ("redmapper_richness_ncounts.fits", True)

In [14]:
ncdata_fits = fits.open('redmapper_richness_ncounts.fits')
ncdata_fits.info()

ncdata_unbinned_data =  ncdata_fits[1].data
ncdata_binned_data = ncdata_fits[2].data

ncdata_unbinned_Table = Table(ncdata_unbinned_data)
display(ncdata_unbinned_Table)

ncdata_binned_Table = Table(ncdata_binned_data)
display(ncdata_binned_Table)

Filename: redmapper_richness_ncounts.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   ()      
  1  ClusterAbundance    1 BinTableHDU     22   38699R x 3C   [1D, 1D, 2D]   
  2  ClusterAbundance    1 BinTableHDU     25   12R x 5C   [1D, 1D, 1D, 1D, 1D]   


Z_OBS,LNM_OBS,Z_OBS_PARAMS [2]
float64,float64,float64
0.513512909412384,190.70664978027344,0.002786155790090561 .. 0.0
0.571269690990448,179.1281280517578,0.00321839633397758 .. 0.0
0.6351852416992188,167.33602905273438,0.00603319751098752 .. 0.0
0.44230303168296814,111.91470336914062,0.0036743979435414076 .. 0.0
0.4669721722602844,104.43976593017578,0.0035005551762878895 .. 0.0
0.4413633942604065,103.49395751953125,0.003672909690067172 .. 0.0
0.5675368309020996,89.5966796875,0.0035968804731965065 .. 0.0
0.500461220741272,77.40795135498047,0.003096552798524499 .. 0.0
0.49930539727211,82.3505630493164,0.003110014135017991 .. 0.0


Z_OBS_LB,Z_OBS_UB,LNM_OBS_LB,LNM_OBS_UB,BIN_COUNT
float64,float64,float64,float64,float64
0.0,0.25,0.0,66.66666666666667,1673.0
0.25,0.5,0.0,66.66666666666667,10159.0
0.5,0.75,0.0,66.66666666666667,14353.0
0.75,1.0,0.0,66.66666666666667,12243.0
0.0,0.25,66.66666666666667,133.33333333333334,14.0
0.25,0.5,66.66666666666667,133.33333333333334,67.0
0.5,0.75,66.66666666666667,133.33333333333334,95.0
0.75,1.0,66.66666666666667,133.33333333333334,69.0
0.0,0.25,133.33333333333334,200.0,3.0
